In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/TO_data/'

Mounted at /content/drive
/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/TO_data


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 
nltk.download('punkt')

from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
from nltk.stem.snowball import SnowballStemmer
import gensim

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
!pip install annoy

     |████████████████████████████████| 655kB 11.3MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp36-cp36m-linux_x86_64.whl size=393334 sha256=5afa7faa7edc6fc8fea8e0a1f4cc88fb01e5fa379a5643eb75bd48017705fdd9
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


**Three Locations**

1) University of Toronto - St. George Campus: <br>
43.66311072883239, -79.39440813687946

2) Scarborough Civic Centre:<br>
43.77952047125285, -79.25780882879529

3) Aga Khan Museum:<br>
43.72803030610192, -79.32977026321979

### Load Data

In [ ]:
def converter(instr):
    return np.fromstring(instr[1:-1],sep=' ')

In [ ]:
#bus = pd.read_csv("business.csv")
ReviewSummary = pd.read_csv("input_data_ver4.csv",converters={'vector':converter})

In [ ]:
bus = pd.read_csv("business.csv")

### Word2Vec Reuslts

In [ ]:
# use simhash to find the most similar items
from annoy import AnnoyIndex

In [ ]:
model = KeyedVectors.load_word2vec_format('word2vec_ver4.vector')

In [ ]:
model.most_similar("italian",topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('napoli', 0.7017148733139038),
 ('neapolitan', 0.6895362138748169),
 ('nonna', 0.6626576781272888),
 ('roman', 0.6599322557449341),
 ('romagna', 0.6585886478424072),
 ('terroni', 0.6448279619216919),
 ('itali', 0.6406687498092651),
 ('neopolitan', 0.6366987228393555),
 ('falasca', 0.6353523135185242),
 ('austrian', 0.6351158022880554)]

In [ ]:
# tokenize data
regEx = re.compile('[^a-z]+')
def tokenize_reviews(text):
  text = text.lower()
  text = regEx.sub(" ", text).strip()
  return text

In [ ]:
# remove stop words
stop_words = set(stopwords.words("english"))
stemmer = SnowballStemmer("english")

def remove_stopwords_stem(text):
  word_tokens = word_tokenize(str(text))
  filtered_sentence = list()
  for w in word_tokens:
    w = regEx.sub(" ", w).strip()
    if w not in stop_words and len(w)>0:
      filtered_sentence.append(stemmer.stem(w))  
  return filtered_sentence

In [ ]:
# get the sentence vector using average value of word vector.
# if a word not in vocabulary, see it as 0
def sentence_vector(sentence):
  vecs = list()
  for word in sentence:
    try:
      vec = model.wv[word]
    except:
      vec = [0] * 100
    vecs.append(vec)
  vector = np.mean(vecs, axis=0)
  return vector

In [ ]:
def keyword_recommend_loc_simhash(input_str, df_old, lat,log):
  lat_min = lat-0.07/2
  lat_max = lat+0.07/2
  log_min = log-0.08/2
  log_max = log+0.08/2 
  bus_id = set(bus[(bus.latitude>=lat_min)&(bus.latitude<=lat_max)&(bus.longitude<=log_max)&(bus.longitude>=log_min)].business_id)
  text_token = tokenize_reviews(input_str)
  text_token_sw_removed = remove_stopwords_stem(text_token)
  vector = sentence_vector(text_token_sw_removed)
  df = df_old[df_old["business_id"].isin(bus_id)]

  # build simhash
  res_vec = list(df.vector)
  f = 100 # dim
  t = AnnoyIndex(f) # length of item vector that will be indexed
  for i in range(len(res_vec)):
    t.add_item(i,res_vec[i])
  t.build(30) # 30 trees

  # Get top 10 recommendations
  indexes = t.get_nns_by_vector(vector, 10)
  result = pd.DataFrame(columns=["business_id"])
  for index in indexes:
    result = result.append({"business_id": df.iloc[index].business_id},ignore_index=True)
  return result

In [ ]:
def top_10_loc_hash(kword, df10,lat,log):
  recommend_list = keyword_recommend_loc_simhash(kword, df10,lat,log)
  return recommend_list

In [ ]:
def cos_similarity(v1,v2):
  return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2))

In [ ]:
# Get the business id information
table = bus[["business_id", "name", "categories", "rating"]]
table.head(3)

,business_id,name,categories,rating
0,EosRKXIGeSWFYWwpkbhNnA,Xtreme Couture,"Martial Arts, Gyms, Fitness & Instruction, Act...",3.0
1,1wWneWD_E1pBIyVpdHMaQg,Air Jordan Store,"Shopping, Shoe Stores, Fashion",4.0
2,007Dg4ESDVacWcC4Vq704Q,Front Door Organics,"Shopping, Food, Organic Stores, Specialty Food...",4.0


In [ ]:
ReviewSummary.head(3)

,business_id,text,summaryClean,review,text_sw_removed,vector
0,-0RRiWDtfnS16AKCtfvBZg,['the independent without an e coffee shop too...,['the independent without an e coffee shop too...,the independent without an e coffee shop took ...,"['independ', 'without', 'e', 'coffe', 'shop', ...","[0.20588197, 0.12743695, 0.1628587, -0.1272542..."
1,-2EAyppKR_2xuLyvJZEbRQ,['i ve tried coming here on separate occasions...,['i ve tried coming here on separate occasions...,i ve tried coming here on separate occasions o...,"['tri', 'come', 'separ', 'occas', 'weekday', '...","[0.19634292, 0.06945179, 0.20247477, -0.055171..."
2,-2TBP3ZGu7M-FmfoNJvbrQ,['we ve been to rasher s several times and hav...,['we ve been to rasher s several times and hav...,we ve been to rasher s several times and have ...,"['rasher', 'sever', 'time', 'realli', 'impress...","[0.18667591, 0.10649319, 0.21303753, -0.034965..."


### Doc2vec Results

In [ ]:
model = gensim.models.Doc2Vec.load('Doc2vec_ver3.vec')  

FileNotFoundError: ignored

### Fried Chicken

**1) University of Toronto - St. George Campus:**<br>
43.66311072883239, -79.39440813687946

In [ ]:
# Word2vec
restaurents = top_10_loc_hash("Fried chicken", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,zsLKMCnwK_NmZJkI7TJk1A,Brando's Fried Chicken,"American (Traditional), Chicken Shop, Restaura...",5.0
1,D4ETxxeNQEnSw_e1H46DXQ,Monga Fried Chicken,"Restaurants, Taiwanese, Fast Food, Asian Fusion",3.5
2,sc-FCrRK6iM28N_N5AKuPw,Popeye's,"Restaurants, Fast Food",3.5
3,Sjgnbhp6arQUM7_7TOzByw,Iron Rooster,"Restaurants, Chicken Wings, Burgers, Fast Food",4.5
4,jrkzgoLCvk_IRw5e-3Zyrw,32 Chicken St.,"Burgers, Asian Fusion, Chicken Shop, Restauran...",4.5
5,U_SKqvm-dUtzDIrixCVdLw,Eating Garden Chinese Restaurant,"Restaurants, Chinese, Food, Specialty Food, Et...",3.0
6,txuns3sXXMKhDFN7JlN8gw,Church's Chicken,"Restaurants, Fast Food, Chicken Wings, Chicken...",4.0
7,Zq9WPT7ZAOYlQ0GV8OQWKQ,Cluck Clucks,"Soul Food, Comfort Food, Sandwiches, Restauran...",4.0
8,e6UDJBEIoXBbnXtdwkqIGA,Hot-Star,"Fast Food, Food, Taiwanese, Chicken Wings, Res...",3.5
9,lI4azrL2urejaJJX-uJ1-A,ChiChop,"Restaurants, Taiwanese, Fast Food, Asian Fusion",3.5


In [ ]:
recommendation.to_csv("FC_loc1.csv",index=False)

**2) Scarborough Civic Centre:**<br>
43.77952047125285, -79.25780882879529

In [ ]:
restaurents = top_10_loc_hash("Fried chicken", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,Rx_YmoVKPWvzGuFuvd-57w,Chick N Delish,"Restaurants, Fast Food, Chicken Wings",3.5
1,HTZFw4QDKhD8KRbnb5ZuJA,Chicken Kitchen,"Restaurants, Chicken Shop, Fast Food, Chicken ...",4.5
2,XZV6pSuJkfsyoT9eJc3dWw,Church's Chicken,"Chicken Shop, Fast Food, Restaurants, Chicken ...",3.0
3,eSSr6I0nt-IZrkq4RgrUbA,Yumys Chicken & Seafood,"Restaurants, Fast Food, Chicken Shop, Seafood",2.0
4,iFWgJ0zmVJ27IeyHcdm1kw,Nugget Pizza and Wings,"Burgers, Chicken Wings, Fast Food, Restaurants...",4.0
5,JTIbsAs3qUq-0bNIh404ng,City Fried Chicken,"Restaurants, Fast Food, Fish & Chips, Cheesest...",4.0
6,ycAW6Q5quaCSDX5zwQ3tPg,New York Fries,"Restaurants, Food, Fast Food, Specialty Food, ...",3.5
7,AIasveRX0245WeAH0C46Uw,Chick Fiesta,"Chicken Shop, South African, African, Portugue...",4.0
8,IfmSqAJ6_IeE1ffQ2AcxsQ,Best Kabab and Karahi,"Indian, Pakistani, Restaurants, Food",3.5
9,my5v0UE6gOukDVmWQB5RpA,Golden Spoon,"Food Delivery Services, Restaurants, Food, Pak...",1.0


In [ ]:
recommendation.to_csv("FC_loc2.csv",index=False)

**Aga Khan Museum**<br>
43.72803030610192, -79.32977026321979

In [ ]:
restaurents = top_10_loc_hash("Fried chicken", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,vlh_e6Ftu2MyS0K-e-wdYA,Chunky Chicken,"Fast Food, Pizza, Chicken Wings, Burgers, Rest...",3.5
1,j4htyBDNBzrog6A_byqdDw,WingBurger,"Comfort Food, Chicken Wings, Food, Fast Food, ...",4.5
2,1XhmwU5PhWABf7IqeLwfBA,4C Broast Chicken,"Restaurants, Halal, Fast Food",4.0
3,ZmnCN-U7mhuZtlWHRsSdtA,Heirloom,"Food, Food Trucks, Restaurants, Burgers, Event...",4.0
4,PMPMHHLe9vFzcR67iCjgLA,Faryab Kabab,"Lebanese, Kebab, Persian/Iranian, Middle Easte...",4.0
5,YIez_A3WOt9J2SXN7OMa2Q,Allwyn's Bakery,"Restaurants, Bakeries, Caribbean, Food",4.0
6,LG7-BUvDLoOZCHDjyiBQFQ,Dirty Grill,"Salad, Restaurants, Juice Bars & Smoothies, Fo...",4.0
7,6U73zDP4NuFKyOQLSigcPA,Spring China House,"Restaurants, Food Delivery Services, Food, Chi...",3.0
8,R9bFr7dADF2fosWJjgbyWQ,Nawzar Meat Market,"Middle Eastern, Meat Shops, Food, Specialty Fo...",4.5
9,dtn4CT4bvZYYfqhGfXpRaw,Cosmos Agora,"Grocery, Bakeries, Specialty Food, Restaurants...",4.5


In [ ]:
recommendation.to_csv("FC_loc3.csv",index=False)

### Sushi

**1) University of Toronto - St. George Campus:**<br>
43.66311072883239, -79.39440813687946

In [ ]:
# Word2vec
restaurents = top_10_loc_hash("Sushi", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,SHa-NH7EDrm61v5nKvVsNg,Fish'D By EDO,"Restaurants, Hawaiian, Food Stands, Japanese",3.0
1,KRWa12MkvE7sgaEnNhSWtQ,Sushi 930,"Sandwiches, Restaurants, Cheese Shops, Food, J...",4.0
2,TKKX9bllRBMzp0NYr441PQ,Mi'Hito Sushi Laboratory,"Food, Poke, Fast Food, Sushi Bars, Restaurants...",4.0
3,jTZYireK6pnPb6Rni0jygw,Wow Sushi,"Food, Restaurants, Sushi Bars",4.0
4,JQIOlyOx7J7XzYzhSkJp9g,Umi Sushi Express,"Restaurants, Fast Food, Japanese",2.0
5,VqReGefofdaspLlQVph2LA,Umi Sushi Express,"Restaurants, Sushi Bars, Food",3.0
6,hXVaXtDO9NnvCOSMtFuHRg,Kawa Sushi,"Food, Food Delivery Services, Restaurants, Jap...",3.5
7,c93AzlUc_j0C-lkWBG92Sw,Sushi Shop,"Food, Japanese, Restaurants, Specialty Food, E...",4.5
8,s4iKGMZeL5Rf-L3yA7l4UA,Tokyo Sushi,"Restaurants, Ethnic Food, Sushi Bars, Food, Sp...",4.5
9,TATCbqRSFNkmC_xNvNmmUA,Yutaka,"Specialty Food, Japanese, Ethnic Food, Restaur...",4.0


In [ ]:
recommendation.to_csv("sushi_loc1.csv",index=False)

**2) Scarborough Civic Centre:**<br>
43.77952047125285, -79.25780882879529

In [ ]:
restaurents = top_10_loc_hash("Sushi", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,aAGo2MMC2hQlG5uhPPNnjA,Su&Shi Noodle Bar,"Restaurants, Noodles, Japanese, Food Court, Su...",3.0
1,Mwl79jLlxPQabA3yiSPz0A,Canton Rice Noodle,"Restaurants, Specialty Food, Food, Chinese, Co...",4.0
2,3FekhBpUmAWXBCvXBhuqVQ,Da Ke Yi Snacks,"Food Court, Chinese, Restaurants",3.5
3,0gj8bpTF6I6T_C5xyBUafw,Arden Fast Food,"Chinese, Food, Restaurants",3.0
4,VnvRZia9M18-IUarfdnYfA,Mucho Burrito,"Mexican, Food, Restaurants, Tex-Mex",3.0
5,B8jsJJMhbASM2IvkU4tmKg,The Rice Kitchen,"Food Delivery Services, Restaurants, Food, Chi...",3.5
6,kiyTiW-2Ba65O3uAMaagTA,Mucho Burrito,"Mexican, Fast Food, Restaurants",2.5
7,6j67nog58__bFPqd620Rrw,JJM Dbarkadz Filipino Cuisine,"Food, Restaurants, Filipino, Food Delivery Ser...",4.0
8,R39q-GVgLaEIh2Ce8PQGDQ,Pizza Nova,"Pizza, Restaurants, Food",2.0
9,1ojeNI9mlgKku1qMRpuMOA,Hong Food,"Asian Fusion, Restaurants, Food Court",3.5


In [ ]:
recommendation.to_csv("sushi_loc2.csv",index=False)

**Aga Khan Museum**<br>
43.72803030610192, -79.32977026321979

In [ ]:
restaurents = top_10_loc_hash("Sushi", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,5sKb4h7R5gdhePSflPD0Tw,Kiku,"Event Planning & Services, Caterers, Restauran...",2.5
1,9jZ9el4v3gWwV2oc5u5kFQ,Pho Saigon,"Food, Restaurants, Vietnamese",3.0
2,6U73zDP4NuFKyOQLSigcPA,Spring China House,"Restaurants, Food Delivery Services, Food, Chi...",3.0
3,yHCoJZh5nx0Onr5xgMaUHg,Galleria Supermarket,"Ethical Grocery, International Grocery, Korean...",4.0
4,FuMc84iDRyPHD8tpkME0yA,Fat Bastard Burrito,"Restaurants, Fast Food, Mexican",2.5
5,GZ12skjXwuMuH3LidRN-mA,Chipotle Mexican Grill,"Restaurants, Spanish, Mexican, Fast Food",3.5
6,bwFxrxHrz9I36awGc-yjjw,TAO Northern Chinese Cuisine,"Dim Sum, Seafood, Chinese, Noodles, Food, Rest...",3.5
7,9xPK9u_AGM1sxoG2CU9psA,Golden Wok Chinese Restaurant,"Chinese, Food, Food Delivery Services, Restaur...",3.5
8,PI2Xr6HwQ5nVUauzSqp6lA,Freshii,"Restaurants, Salad, Fast Food, Health Markets,...",2.5
9,5zM4mYdaRmchdQeo8dVw3g,C&C Supermarket,"Specialty Food, Restaurants, Food, Seafood Mar...",4.0


In [ ]:
recommendation.to_csv("sushi_loc3.csv",index=False)

### Bakery

In [ ]:
# University of Toronto - St. George Campus:
# Word2vec
restaurents = top_10_loc_hash("Bakery", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,IflMVeq2zfidJrf-D6AcZw,Cafe,"Patisserie/Cake Shop, Food, Gluten-Free, Cafes...",3.5
1,PGJd7yyMitAgYRRYvxxEHA,Venezia Bakery,"Restaurants, Bakeries, Food, Delis, Portuguese",4.0
2,ZALVCrIRwuvTrQlPnSHdVw,Two Peas,"Shopping, Coffee & Tea, Cafes, Hostels, Food, ...",4.0
3,j2U2vs4WWlF6SR5SS_D1lA,Brazil Bakery and Pastry,"Restaurants, Bakeries, Brazilian, Food, Portug...",4.0
4,Cx5oz2bHHa62W248CR1B4w,Alcoa Bakery,"Bakeries, Portuguese, Restaurants, Food",4.0
5,0PlOgLqjGxnTC484RjHpIg,Think It Pensa Italiano,"Cafes, Coffee & Tea, Food, Italian, Restaurant...",4.5
6,JgABahalY1X5qAR5JGxpRA,Roselle West,"Cafes, Restaurants, Food, Desserts, Bakeries",4.5
7,9OD0pQ0QYa_7dO-LbOBFZQ,Furama Cake and Desserts Garden,"Restaurants, Chinese, Food, Bakeries",3.0
8,DlhhSd13HsuRqPT6h3kvAg,Harbord Bakery,"Kosher, Food, Bakeries, Restaurants",3.5
9,tVqo5i1m2tOIoGCfP8CpZg,Bloomer’s,"Sandwiches, Bakeries, Vegan, Food, Cafes, Rest...",5.0


In [ ]:
recommendation.to_csv("bakery_loc1.csv",index=False)

In [ ]:
# Scarborough Civic Centre
restaurents = top_10_loc_hash("Bakery", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,upbxmoNqlovW21O7bibV4Q,Aroma Bakery,"Chinese, Restaurants, Bakeries, Dim Sum, Food",4.5
1,g7EMn2tfrmAKBW9rQQeeEw,Ladan Pastry & Bakery,"Restaurants, Persian/Iranian, Food, Bakeries",4.0
2,sA-YqTMe2bJmg1MgbfowuA,Whisked Gluten-Free,"Bakeries, Desserts, Food, Restaurants, Gluten-...",4.5
3,Koigz4oP9YnHlPhBhqXCxA,New Dynasty Bakery,"Restaurants, Food, Chinese, Bakeries",2.0
4,FS8uGqtkLVl5P5iPhTAGjQ,Robin's,"Juice Bars & Smoothies, Restaurants, Cafes, Fo...",3.5
5,CINuanTB3t119JypJq0DTw,Lebanese Bakery,"Desserts, Food, Restaurants, Middle Eastern, B...",3.0
6,4J5WT8HM2IINOmVtvRihbg,Kin Kin Bubble Tea,"Cafes, Bakeries, Food, Restaurants",3.0
7,rr9sk5DxL8wHxqIF48uJlg,Patisserie Royale,"Restaurants, Bakeries, Food, Middle Eastern",4.5
8,mUUTJY9_BqmAYxypHiqWrw,Francesca,"Food, Bakeries, Desserts, Italian, Coffee & Te...",4.0
9,C4X26YL0fNu5CuW8Flyo0g,Crème et Miel,"Restaurants, Juice Bars & Smoothies, Lebanese,...",4.5


In [ ]:
recommendation.to_csv("bakery_loc2.csv",index=False)

In [ ]:
# Aga Khan Museum
restaurents = top_10_loc_hash("Bakery", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,g7EMn2tfrmAKBW9rQQeeEw,Ladan Pastry & Bakery,"Restaurants, Persian/Iranian, Food, Bakeries",4.0
1,FS8uGqtkLVl5P5iPhTAGjQ,Robin's,"Juice Bars & Smoothies, Restaurants, Cafes, Fo...",3.5
2,fwXKszT6j_f6pf0tp3MHLw,Patisserie Royale,"Bakeries, Food, Middle Eastern, Restaurants",5.0
3,LSneBLmL_mLRKjplp2BVZg,ABC Euro Delicatessen,"Delicatessen, Food, Grocery, Ethnic Grocery, D...",4.0
4,5zM4mYdaRmchdQeo8dVw3g,C&C Supermarket,"Specialty Food, Restaurants, Food, Seafood Mar...",4.0
5,C4X26YL0fNu5CuW8Flyo0g,Crème et Miel,"Restaurants, Juice Bars & Smoothies, Lebanese,...",4.5
6,m_rvCLB8j10NeUZPzuK1fA,Crown Pastries,"Bakeries, Food, Desserts, Middle Eastern, Rest...",4.5
7,9rvRHoEgbUDAx4jbzv80uQ,Memories of Africa,"Restaurants, Ethnic Food, Food, African, Speci...",2.5
8,zNCGTEwBrmrVLfjxl7LenQ,Ballissimo Loukoumades Bar,"Desserts, Cafes, Food, Restaurants",5.0
9,Ro_--o91peOp3KFXUieXcA,Tim Hortons,"Restaurants, Food, Coffee & Tea, Donuts, Cafes",2.0


In [ ]:
recommendation.to_csv("bakery_loc3.csv",index=False)

### Bar

In [ ]:
# University of Toronto - St. George Campus:
# Word2vec
restaurents = top_10_loc_hash("Bar", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,IZ0c8VGiDeURttERwuxuqg,ABBIOCCO,"Nightlife, Seafood, French, Restaurants, Wine ...",1.0
1,ByAj0FqbFYQhkZYboEnfFg,Quincy's Afro-Kitchen and Bar,"Restaurants, Ethnic Food, African, Nightlife, ...",3.0
2,FVgaLPmq88zX0mT5NJPFgw,Alice's Place,"Pubs, Bars, Karaoke, Nightlife, Fast Food, Res...",3.5
3,MMgOuyfZQtQQJsLPYtqE1Q,Lola,"Food, Nightlife, Dive Bars, Restaurants, Canad...",3.0
4,g1sp3BMnNfBiVBvZy_8V6g,Gift Shop,"Food, Juice Bars & Smoothies, Cocktail Bars, R...",3.5
5,n9eudXq_35C0htRLtHZ2XQ,Bathurst Local,"Whiskey Bars, Beer Bar, Food, Coffee & Tea, Co...",4.5
6,fNJRiDAT6Qy9PQ3nqA_L9g,Concord Cafe,"Restaurants, Food, Coffee & Tea, Latin America...",3.0
7,-bJ6Y5QmO8Uvc383xjC-OA,The Monarch Tavern,"Lounges, Music Venues, Restaurants, Pubs, Bars...",4.0
8,q7MorRPzU_J-iekeDKUKgw,Reposado Bar & Lounge,"Restaurants, Food, Jazz & Blues, Cocktail Bars...",4.0
9,2dAlfY0SiQ997134xzy43Q,Pixels and Pints,"Restaurants, Bubble Tea, Nightlife, Bars, Arts...",3.5


In [ ]:
recommendation.to_csv("bar_loc1.csv",index=False)

In [ ]:
# Scarborough Civic Centre
restaurents = top_10_loc_hash("Bar", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,C4X26YL0fNu5CuW8Flyo0g,Crème et Miel,"Restaurants, Juice Bars & Smoothies, Lebanese,...",4.5
1,FS8uGqtkLVl5P5iPhTAGjQ,Robin's,"Juice Bars & Smoothies, Restaurants, Cafes, Fo...",3.5
2,z9_vbTmxItQ_qUn0mo-Nyw,Helious Cafe,"Cafes, Restaurants, Food, Juice Bars & Smoothi...",2.0
3,q-E9oE0StdOfMWIHiotfng,Coffee Culture,"Breakfast & Brunch, Restaurants, Coffee & Tea,...",2.5
4,n-UVXXRQNVEiP6CRrmRAzw,The Mountain Grill and Juice Bar,"Food, Restaurants, Mediterranean, Playgrounds,...",2.5
5,fJAWn77rOLON08wli_AQ2Q,Burger King,"Burgers, Restaurants, Fast Food",4.0
6,NR2-lpZ91nDvP5pfavhtIQ,Tea A Break Tea House,"Chinese, Restaurants, Food, Tea Rooms",2.5
7,SnpjfcOZyClpdsBDTr7zfg,Juicy Teahouse,"Restaurants, Juice Bars & Smoothies, Coffee & ...",3.0
8,URGUzMJRhK5gLQKC4UztTQ,Taco bell,"Fast Food, Mexican, Restaurants",3.0
9,KvfaPsLlpOZAF276Lpvm4w,Tealicious Tea House,"Chinese, Taiwanese, Coffee & Tea, Bubble Tea, ...",3.0


In [ ]:
recommendation.to_csv("bar_loc2.csv",index=False)

In [ ]:
# Aga Khan Museum
restaurents = top_10_loc_hash("Bar", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,bY-4nSmbowNdAdrch0P9Tw,Corks Beer & Wine Bars,"Restaurants, Food, Nightlife, Bars, Canadian (...",4.5
1,iQDPdxmXXvW-mgy9gsKXhQ,Starbucks Reserve,"Restaurants, Coffee Roasteries, Coffee & Tea, ...",4.0
2,EEu0F38DozQCmhDkh9g6cw,The Mill,"Restaurants, Nightlife, American (Traditional)...",3.5
3,pnStRO16rw-jXa9S2FQoEg,Alexandria Cafe,"Donairs, Middle Eastern, Food, Restaurants, Ho...",2.0
4,bCzqTllyOJoH3S7e_AHJNw,The Moose And Barrel Pub & Restaurant,"Nightlife, Restaurants, Food, Beer, Wine & Spi...",2.5
5,C4X26YL0fNu5CuW8Flyo0g,Crème et Miel,"Restaurants, Juice Bars & Smoothies, Lebanese,...",4.5
6,URvh6DgFyA-d6Z7OVKxczg,Rally Sports Bar + Smokehouse,"Smokehouse, Food, Barbeque, Nightlife, Comfort...",4.0
7,FS8uGqtkLVl5P5iPhTAGjQ,Robin's,"Juice Bars & Smoothies, Restaurants, Cafes, Fo...",3.5
8,iLPDJsSBF5n_YGBJu3bgOQ,Starbucks,"Food, Coffee & Tea, Sandwiches, Restaurants, I...",3.0
9,ZTrkiSudc8vL3NB2N7XFJw,Nostalgia Coffee Company,"Sandwiches, Restaurants, Coffee & Tea, Food, B...",4.0


In [ ]:
recommendation.to_csv("bar_loc3.csv",index=False)

### steak house

In [ ]:
# University of Toronto - St. George Campus:
# Word2vec
restaurents = top_10_loc_hash("steak house", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,nLBYO2uLPHzZ9pceGmlwNA,Druxy's Famous Deli,"Breakfast & Brunch, Delicatessen, Sandwiches, ...",2.0
1,JWMNEt7Xhmi15_B-DaDHhw,Nobs',"Sandwiches, Street Vendors, Food, Restaurants,...",4.0
2,CXn6LpyfwucwM-DfMw0oPw,The Charcoal Grill,"Food, Desserts, Restaurants, Sandwiches, Barbeque",4.0
3,s11Ym0QX2Am2uwtnyou4GA,Quinta Restaurant,"Iberian, Food, Restaurants, American (Traditio...",4.0
4,T4g4k2OhCTsiqcP-ZXElHQ,Jules Bistro,"Restaurants, Food, French",4.0
5,ZhvYzRAjoQYW8ONFRg1tlg,Buster Rhino's Southern BBQ,"Food, Barbeque, Restaurants",3.0
6,qeIKP9mGEucrdxbGo9uRTw,Bourbon Street Smokehouse,"Barbeque, Restaurants, Southern, Food, Smokehouse",4.0
7,8cusyHsqMhNAsrNSMpzB1A,The 420 Smokehouse,"Food, Smokehouse, American (Traditional), Barb...",4.0
8,8AAHDz9VIAvq8XbKJHrSrg,Per Se Mobile,"Food Trucks, Food, Restaurants, Food Stands",4.0
9,bbzU5h4Fr9zDac1KrS-_2Q,The Saucy Pierogi,"Pop-up Shops, Caterers, Shopping, Event Planni...",4.0


In [ ]:
recommendation.to_csv("sh_loc1.csv",index=False)

In [ ]:
# Scarborough Civic Centre
restaurents = top_10_loc_hash("steak house", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,dtn4CT4bvZYYfqhGfXpRaw,Cosmos Agora,"Grocery, Bakeries, Specialty Food, Restaurants...",4.5
1,JTIbsAs3qUq-0bNIh404ng,City Fried Chicken,"Restaurants, Fast Food, Fish & Chips, Cheesest...",4.0
2,lz9oUUVaPJsC5aaU_6nzsQ,Fresh Choice Cafe,"Restaurants, Coffee & Tea, Food, Cafes, Delis",4.5
3,dQojkooHaWvyCor5qcAS6Q,Peking Duck Kitchen,"Specialty Food, Food, Restaurants, Chinese",3.5
4,d3JhScywFHns4gdR7vIqlA,The Wing Spot,"Restaurants, Chicken Wings, American (Traditio...",4.0
5,GKEl7LNGUqJiQWNQ0vpH8Q,Milestones Restaurants,"American (Traditional), Bars, Restaurants, Com...",3.0
6,iFWgJ0zmVJ27IeyHcdm1kw,Nugget Pizza and Wings,"Burgers, Chicken Wings, Fast Food, Restaurants...",4.0
7,AIasveRX0245WeAH0C46Uw,Chick Fiesta,"Chicken Shop, South African, African, Portugue...",4.0
8,a7mTbEi2N8Zd-r-8jlReww,Koryo Korean Barbeque,"Barbeque, Japanese, Korean, Fast Food, Restaur...",3.5
9,Rx_YmoVKPWvzGuFuvd-57w,Chick N Delish,"Restaurants, Fast Food, Chicken Wings",3.5


In [ ]:
recommendation.to_csv("sh_loc2.csv",index=False)

In [ ]:
# Aga Khan Museum
restaurents = top_10_loc_hash("steak house", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,LG7-BUvDLoOZCHDjyiBQFQ,Dirty Grill,"Salad, Restaurants, Juice Bars & Smoothies, Fo...",4.0
1,dtn4CT4bvZYYfqhGfXpRaw,Cosmos Agora,"Grocery, Bakeries, Specialty Food, Restaurants...",4.5
2,fbBTvnuHPMsAQs_RShdVyw,Grilltime Gourmet Meat Shop,"Specialty Food, Food, Meat Shops, Grocery, Res...",4.0
3,PMPMHHLe9vFzcR67iCjgLA,Faryab Kabab,"Lebanese, Kebab, Persian/Iranian, Middle Easte...",4.0
4,ExoqgA3aAz6bYidI0DSQvw,California Sandwiches,"Restaurants, Food, Sandwiches",4.0
5,R9bFr7dADF2fosWJjgbyWQ,Nawzar Meat Market,"Middle Eastern, Meat Shops, Food, Specialty Fo...",4.5
6,EwHgw3rDJiqU3XDJvcpR8w,Subway,"Sandwiches, Restaurants, Fast Food",2.5
7,ZmnCN-U7mhuZtlWHRsSdtA,Heirloom,"Food, Food Trucks, Restaurants, Burgers, Event...",4.0
8,51lFCPpAZz4t7-BrpLHf5g,Henry's Chicken,"Restaurants, Peruvian, Comfort Food, Chicken S...",4.0
9,Ko-GLxKpNikHBABYhEtUjg,Topham Corner,"Nightlife, Arts & Entertainment, Jazz & Blues,...",4.5


In [ ]:
recommendation.to_csv("sh_loc3.csv",index=False)

### Chinese Food

In [ ]:
# University of Toronto - St. George Campus:
# Word2vec
restaurents = top_10_loc_hash("Chinese Food", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,qoDrPDPEGRsF8I_VgYftjQ,Wikki Hut,"Fast Food, Food Court, Restaurants, Salad",3.5
1,9pBcKw24z9nZiHW7Vx_z8A,Manchu Wok,"Restaurants, Fast Food, Chinese, Asian Fusion",3.5
2,fuk8V-SmFO0xRyC9eMDarw,China Garden Restaurant,"Indian, Chinese, Food, Specialty Food, Restaur...",2.5
3,DEdB-wkPV1QzcSFMTTYl-w,South Sea Chinese Foods,"Ethnic Food, Food, Chinese, Restaurants, Speci...",3.0
4,FLTgadyh5s8JIBwgR_xxyA,Ceylon Spice by Esther,"Food Court, Restaurants, Indian, Sri Lankan",3.0
5,n-ps87_nlV56dKLD7Wep3w,Fookee Express,"Chinese, Fast Food, Restaurants",3.0
6,D2loJREaIqjam3ct5q2Hcg,China Wok,"Chinese, Restaurants, Fast Food, Food Court",4.5
7,OaO_5YeBF9xBXmcgG1t0WA,Indochine Thai Vietnamese Restaurant,"Vietnamese, Thai, Food, Restaurants, Specialty...",3.0
8,SykyXR8BFiBRH9ON-Kj9kg,Shanghai 360,"Food Court, Fast Food, Soup, Restaurants, Dim ...",3.0
9,hqpPDu238T1tpBa64Q0TSw,Tasty Chinese Food,"Restaurants, Chinese, Ethnic Food, Specialty F...",3.0


In [ ]:
recommendation.to_csv("cf_loc1.csv",index=False)

In [ ]:
# Scarborough Civic Centre
restaurents = top_10_loc_hash("Chinese Food", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,0gj8bpTF6I6T_C5xyBUafw,Arden Fast Food,"Chinese, Food, Restaurants",3.0
1,1ojeNI9mlgKku1qMRpuMOA,Hong Food,"Asian Fusion, Restaurants, Food Court",3.5
2,B8jsJJMhbASM2IvkU4tmKg,The Rice Kitchen,"Food Delivery Services, Restaurants, Food, Chi...",3.5
3,PCdQZQFvcjeiiXW_W0HBKA,Kenny's Noodles,"Ethnic Food, Chinese, Specialty Food, Food, Re...",4.5
4,ZLPPaAJnEvN8311zwXvw6w,Touch Down Foods,"Desserts, Grocery, Restaurants, Imported Food,...",5.0
5,5E2bJVdz5Qk-TJlFeawI7Q,Taste of Hong Kong,"Food Court, Restaurants, Hong Kong Style Cafe",3.5
6,1kqLZp0_FEMHGx-W4TjTLQ,Hong Kong Style Tea & Fast Food,"Fast Food, Restaurants, Chinese, Food Court",4.0
7,Exsi3PIwpfvuoyF8BfGYIQ,HaveMore,"Food Court, Indian, Caterers, Event Planning &...",4.0
8,3FekhBpUmAWXBCvXBhuqVQ,Da Ke Yi Snacks,"Food Court, Chinese, Restaurants",3.5
9,A0iy8p1nlL4Jt0Vssss3yQ,Supreme Restaurant & Bar,"Restaurants, Chinese, Soul Food, Nightlife, Ba...",3.0


In [ ]:
recommendation.to_csv("cf_loc2.csv",index=False)

In [ ]:
# Aga Khan Museum
restaurents = top_10_loc_hash("Chinese Food", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,6U73zDP4NuFKyOQLSigcPA,Spring China House,"Restaurants, Food Delivery Services, Food, Chi...",3.0
1,bwFxrxHrz9I36awGc-yjjw,TAO Northern Chinese Cuisine,"Dim Sum, Seafood, Chinese, Noodles, Food, Rest...",3.5
2,xUsYf7lB0bi1zGEw41-aTw,Stoke Stack Barbecue,"Food, Barbeque, Food Trucks, Restaurants",5.0
3,KOQF_A4XrRrWYBWl5aSZ7w,Chakula Hut,"Halal, African, Comfort Food, Restaurants, Ind...",3.5
4,9jZ9el4v3gWwV2oc5u5kFQ,Pho Saigon,"Food, Restaurants, Vietnamese",3.0
5,9xPK9u_AGM1sxoG2CU9psA,Golden Wok Chinese Restaurant,"Chinese, Food, Food Delivery Services, Restaur...",3.5
6,5zM4mYdaRmchdQeo8dVw3g,C&C Supermarket,"Specialty Food, Restaurants, Food, Seafood Mar...",4.0
7,nGCKQlzat6nEKTImlpg_hg,Joy Fortune Chinese Cuisine,"Beer, Wine & Spirits, Food, Chinese, Restaurants",3.5
8,GoOkCxPFM5i5lxHgFQMj1w,Asian Legend,"Chinese, Restaurants, Food",3.0
9,yHCoJZh5nx0Onr5xgMaUHg,Galleria Supermarket,"Ethical Grocery, International Grocery, Korean...",4.0


In [ ]:
recommendation.to_csv("cf_loc3.csv",index=False)

### Indian Food

In [ ]:
# University of Toronto - St. George Campus:
# Word2vec
restaurents = top_10_loc_hash("Indian Food", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,FLTgadyh5s8JIBwgR_xxyA,Ceylon Spice by Esther,"Food Court, Restaurants, Indian, Sri Lankan",3.0
1,qoDrPDPEGRsF8I_VgYftjQ,Wikki Hut,"Fast Food, Food Court, Restaurants, Salad",3.5
2,9pBcKw24z9nZiHW7Vx_z8A,Manchu Wok,"Restaurants, Fast Food, Chinese, Asian Fusion",3.5
3,DEdB-wkPV1QzcSFMTTYl-w,South Sea Chinese Foods,"Ethnic Food, Food, Chinese, Restaurants, Speci...",3.0
4,Ijh1AcbH-F3CyhYT_5qL8Q,Indian Grill,"Food, Restaurants, Indian",4.0
5,muInTkijds_uH2uFzNMaNw,Spice House Indian Cuisine,"Food, Restaurants, Indian",4.5
6,sJ0MYSAIVK28cMzh-s-NPA,Amaya Express,"Fast Food, Indian, Restaurants",2.5
7,fuk8V-SmFO0xRyC9eMDarw,China Garden Restaurant,"Indian, Chinese, Food, Specialty Food, Restaur...",2.5
8,xqNpOnxY0XzdnUE4sjmg0w,Chaska,"Grocery, Food, Indian, Coffee & Tea, Restaurants",4.0
9,vZu3xxlMQSqFngDgdFLfXg,Tamarind - The Indian Kitchen,"Fast Food, Vegetarian, Restaurants, Indian",3.5


In [ ]:
recommendation.to_csv("If_loc1.csv",index=False)

In [ ]:
# Scarborough Civic Centre
restaurents = top_10_loc_hash("Indian Food", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,Exsi3PIwpfvuoyF8BfGYIQ,HaveMore,"Food Court, Indian, Caterers, Event Planning &...",4.0
1,ZLPPaAJnEvN8311zwXvw6w,Touch Down Foods,"Desserts, Grocery, Restaurants, Imported Food,...",5.0
2,2vyjdKZJM5fv0v7Iob0FcQ,Mirchi Dhaba,"Food, Imported Food, Do-It-Yourself Food, Spec...",3.0
3,A0iy8p1nlL4Jt0Vssss3yQ,Supreme Restaurant & Bar,"Restaurants, Chinese, Soul Food, Nightlife, Ba...",3.0
4,1ojeNI9mlgKku1qMRpuMOA,Hong Food,"Asian Fusion, Restaurants, Food Court",3.5
5,C_chktl5jFeRqu1C0rmScA,Kairali - Taste of Kerala,"Restaurants, Indian, Food",3.5
6,0gj8bpTF6I6T_C5xyBUafw,Arden Fast Food,"Chinese, Food, Restaurants",3.0
7,PCdQZQFvcjeiiXW_W0HBKA,Kenny's Noodles,"Ethnic Food, Chinese, Specialty Food, Food, Re...",4.5
8,my5v0UE6gOukDVmWQB5RpA,Golden Spoon,"Food Delivery Services, Restaurants, Food, Pak...",1.0
9,KErZUJuBlf-Abb_xB4xVIw,Bombay Bites,"Vegetarian, Restaurants, Indian, Fast Food",2.5


In [ ]:
recommendation.to_csv("If_loc2.csv",index=False)

In [ ]:
# Aga Khan Museum
restaurents = top_10_loc_hash("Indian Food", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,6U73zDP4NuFKyOQLSigcPA,Spring China House,"Restaurants, Food Delivery Services, Food, Chi...",3.0
1,KOQF_A4XrRrWYBWl5aSZ7w,Chakula Hut,"Halal, African, Comfort Food, Restaurants, Ind...",3.5
2,xUsYf7lB0bi1zGEw41-aTw,Stoke Stack Barbecue,"Food, Barbeque, Food Trucks, Restaurants",5.0
3,oo73pe5tERpDxaa_eSY2tQ,Iqbal Kebab & Sweet Centre,"Restaurants, Barbeque, Grocery, Seafood, Pakis...",4.0
4,MTNDAZ5PrnflhmdyKqVJ8w,Kashgar Kebab House,"Middle Eastern, Restaurants, Food, Bakeries, M...",4.0
5,9jZ9el4v3gWwV2oc5u5kFQ,Pho Saigon,"Food, Restaurants, Vietnamese",3.0
6,9rvRHoEgbUDAx4jbzv80uQ,Memories of Africa,"Restaurants, Ethnic Food, Food, African, Speci...",2.5
7,-J6FVdY9pSgAdFmmalO-pQ,Jatujak,"Food, Thai, Food Delivery Services, Restaurants",4.0
8,bwFxrxHrz9I36awGc-yjjw,TAO Northern Chinese Cuisine,"Dim Sum, Seafood, Chinese, Noodles, Food, Rest...",3.5
9,PMPMHHLe9vFzcR67iCjgLA,Faryab Kabab,"Lebanese, Kebab, Persian/Iranian, Middle Easte...",4.0


In [ ]:
recommendation.to_csv("If_loc3.csv",index=False)

### Bubble tea

In [ ]:
# University of Toronto - St. George Campus:
# Word2vec
restaurents = top_10_loc_hash("Bubble tea", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,LMyeQgw0qO91iCnkltDI4A,teashop 168,"Taiwanese, Cafes, Food, Coffee & Tea, Restaurants",3.5
1,kDuZOo-tVIvIbD5zVBxHuA,Gong Cha,"Juice Bars & Smoothies, Taiwanese, Food, Bubbl...",4.0
2,pRwh3BEM-dhixAGJl_TgUA,Ten Ren Tea Company,"Restaurants, Chinese, Coffee & Tea, Food",2.5
3,Auj0qSYqgt-T5Gd0HhA2Rw,Bubble Tease,"Restaurants, Chinese, Tea Rooms, Juice Bars & ...",3.5
4,snRxtJLoxm5_JdNURJXEUg,Cuppa Tea,"Restaurants, Bubble Tea, Waffles, Food",4.5
5,nc2ToNeakcNlFOggSuh0SQ,Comebuy Teashop,"Taiwanese, Tea Rooms, Restaurants, Coffee & Te...",3.5
6,Cbvx1IxPniwo_Ybk4CdIBQ,Tao Tea Leaf,"Food, Restaurants, Tea Rooms, Chinese",4.5
7,i6gKYG_YFuF5o90jPuWYvw,CoCo Fresh Juice & Tea,"Food, Juice Bars & Smoothies, Cafes, Tea Rooms...",4.0
8,ZDDzKXN_JXe7694zePRuTQ,Chatime - Dundas,"Chinese, Restaurants, Taiwanese, Tea Rooms, Et...",3.5
9,A49jCMzZXswmSAZqZUAi8Q,E-Tea - Bubble Tea and Singaporean Cuisine,"Coffee & Tea, Singaporean, Food, Restaurants",3.0


In [ ]:
recommendation.to_csv("bb_loc1.csv",index=False)

In [ ]:
# Scarborough Civic Centre
restaurents = top_10_loc_hash("Bubble tea", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,zu5dqKqPjJCycS5k-92Alg,Chaplus,"Waffles, Tea Rooms, Restaurants, Coffee & Tea,...",3.0
1,kBYBXh7M9wKXvHgluJWiLQ,Annie's Tea House,"Food Stands, Juice Bars & Smoothies, Bubble Te...",4.0
2,ii_zDR8crbQVJehMMWoCkQ,Marathon Donuts & Coffee Shop,"Breakfast & Brunch, Restaurants, Food, Coffee ...",3.5
3,ZPI1t-WcZruILq7OQYnZHg,Real Fruit Bubble Tea,"Juice Bars & Smoothies, Food, Tea Rooms, Asian...",4.5
4,SnpjfcOZyClpdsBDTr7zfg,Juicy Teahouse,"Restaurants, Juice Bars & Smoothies, Coffee & ...",3.0
5,KvfaPsLlpOZAF276Lpvm4w,Tealicious Tea House,"Chinese, Taiwanese, Coffee & Tea, Bubble Tea, ...",3.0
6,57CQrcshejFYdEncv--zLg,Chill Station,"Chinese, Food, Bubble Tea, Restaurants, Asian ...",3.0
7,LN2X2X2PVsd56tqLIEku9Q,Chill,"Restaurants, Coffee & Tea, Food, Chinese, Dess...",4.0
8,4J5WT8HM2IINOmVtvRihbg,Kin Kin Bubble Tea,"Cafes, Bakeries, Food, Restaurants",3.0
9,NR2-lpZ91nDvP5pfavhtIQ,Tea A Break Tea House,"Chinese, Restaurants, Food, Tea Rooms",2.5


In [ ]:
recommendation.to_csv("bb_loc2.csv",index=False)

In [ ]:
# Aga Khan Museum
restaurents = top_10_loc_hash("Bubble tea", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,5CSCTpBZYl_uDN7ZjSp5yQ,Real Fruit Bubble Tea,"Food Court, Restaurants, Food, Juice Bars & Sm...",2.5
1,vOdY5xomz8nNX756zNYQxQ,Real Fruit Bubble Tea,"Restaurants, Taiwanese, Food, Bubble Tea, Coff...",2.5
2,C4X26YL0fNu5CuW8Flyo0g,Crème et Miel,"Restaurants, Juice Bars & Smoothies, Lebanese,...",4.5
3,iQDPdxmXXvW-mgy9gsKXhQ,Starbucks Reserve,"Restaurants, Coffee Roasteries, Coffee & Tea, ...",4.0
4,1hUvpPDLiVS7Ht2LPROFTA,Demetres Shops At Don Mills,"Cafes, Coffee & Tea, Desserts, Food, Ice Cream...",4.0
5,iLPDJsSBF5n_YGBJu3bgOQ,Starbucks,"Food, Coffee & Tea, Sandwiches, Restaurants, I...",3.0
6,FS8uGqtkLVl5P5iPhTAGjQ,Robin's,"Juice Bars & Smoothies, Restaurants, Cafes, Fo...",3.5
7,Ro_--o91peOp3KFXUieXcA,Tim Hortons,"Restaurants, Food, Coffee & Tea, Donuts, Cafes",2.0
8,KZoFzDi6YXUG8kH_LpOw3w,JJ Bean,"Coffee & Tea, Cafes, Bakeries, Coffee Roasteri...",5.0
9,zNCGTEwBrmrVLfjxl7LenQ,Ballissimo Loukoumades Bar,"Desserts, Cafes, Food, Restaurants",5.0


In [ ]:
recommendation.to_csv("bb_loc3.csv",index=False)

### Mexican Food

In [ ]:
# University of Toronto - St. George Campus:
# Word2vec
restaurents = top_10_loc_hash("Mexican Food", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,qoDrPDPEGRsF8I_VgYftjQ,Wikki Hut,"Fast Food, Food Court, Restaurants, Salad",3.5
1,17HGaSwTNL-9JKWi8oAgqA,Taco Bell,"Mexican, American (Traditional), Restaurants, ...",3.0
2,FLTgadyh5s8JIBwgR_xxyA,Ceylon Spice by Esther,"Food Court, Restaurants, Indian, Sri Lankan",3.0
3,1W_gZM_uuEvJqapbIl6z9Q,El Charro,"Food Stands, Food, Mexican, Street Vendors, Re...",4.0
4,YiFCHXw7oZQg53oM3e0R4g,Mucho Burrito,"Fast Food, Mexican, Tex-Mex, Restaurants",2.5
5,NqOmAGGEed-W4eXWlMd0Bw,Taco Bell,"Restaurants, Mexican, Tex-Mex, Fast Food",3.5
6,n_4PTgvjsKc74u8AwXOfAQ,Mexican Salsas,"Gluten-Free, Restaurants, Fast Food, Mexican",4.0
7,9pBcKw24z9nZiHW7Vx_z8A,Manchu Wok,"Restaurants, Fast Food, Chinese, Asian Fusion",3.5
8,lAYp-_CGc9K_azlDC6epBg,Tacofest,"Arts & Entertainment, Restaurants, Festivals, ...",3.0
9,BhUDq-HSrqMFfR03V2FoQQ,Rancho Relaxo To Go,"Food, Mexican, Food Trucks, Restaurants, Veget...",3.5


In [ ]:
recommendation.to_csv("mf_loc1.csv",index=False)

In [ ]:
# Scarborough Civic Centre
restaurents = top_10_loc_hash("Mexican Food", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,URGUzMJRhK5gLQKC4UztTQ,Taco bell,"Fast Food, Mexican, Restaurants",3.0
1,VnvRZia9M18-IUarfdnYfA,Mucho Burrito,"Mexican, Food, Restaurants, Tex-Mex",3.0
2,ZLPPaAJnEvN8311zwXvw6w,Touch Down Foods,"Desserts, Grocery, Restaurants, Imported Food,...",5.0
3,1ojeNI9mlgKku1qMRpuMOA,Hong Food,"Asian Fusion, Restaurants, Food Court",3.5
4,PCdQZQFvcjeiiXW_W0HBKA,Kenny's Noodles,"Ethnic Food, Chinese, Specialty Food, Food, Re...",4.5
5,0gj8bpTF6I6T_C5xyBUafw,Arden Fast Food,"Chinese, Food, Restaurants",3.0
6,A0iy8p1nlL4Jt0Vssss3yQ,Supreme Restaurant & Bar,"Restaurants, Chinese, Soul Food, Nightlife, Ba...",3.0
7,B8jsJJMhbASM2IvkU4tmKg,The Rice Kitchen,"Food Delivery Services, Restaurants, Food, Chi...",3.5
8,LwH0uzsl_Rsb5MsJqaNc6w,Fat Bastard Burrito,"Restaurants, Mexican, Fast Food",3.0
9,kiyTiW-2Ba65O3uAMaagTA,Mucho Burrito,"Mexican, Fast Food, Restaurants",2.5


In [ ]:
recommendation.to_csv("mf_loc2.csv",index=False)

In [ ]:
# Aga Khan Museum
restaurents = top_10_loc_hash("Mexican Food", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,xUsYf7lB0bi1zGEw41-aTw,Stoke Stack Barbecue,"Food, Barbeque, Food Trucks, Restaurants",5.0
1,KOQF_A4XrRrWYBWl5aSZ7w,Chakula Hut,"Halal, African, Comfort Food, Restaurants, Ind...",3.5
2,6U73zDP4NuFKyOQLSigcPA,Spring China House,"Restaurants, Food Delivery Services, Food, Chi...",3.0
3,GZ12skjXwuMuH3LidRN-mA,Chipotle Mexican Grill,"Restaurants, Spanish, Mexican, Fast Food",3.5
4,A9PWiKkXaw6TsC9U-i4i-w,Mustache Burger,"Food Trucks, Restaurants, Burgers, Food",2.5
5,9rvRHoEgbUDAx4jbzv80uQ,Memories of Africa,"Restaurants, Ethnic Food, Food, African, Speci...",2.5
6,PI2Xr6HwQ5nVUauzSqp6lA,Freshii,"Restaurants, Salad, Fast Food, Health Markets,...",2.5
7,9jZ9el4v3gWwV2oc5u5kFQ,Pho Saigon,"Food, Restaurants, Vietnamese",3.0
8,EEu0F38DozQCmhDkh9g6cw,The Mill,"Restaurants, Nightlife, American (Traditional)...",3.5
9,3Z55ll2Rx43I96aozFG3Eg,Pizza Land,"Restaurants, Fast Food, Pizza",3.5


In [ ]:
recommendation.to_csv("mf_loc3.csv",index=False)

### seafood


In [ ]:
# University of Toronto - St. George Campus:
# Word2vec
restaurents = top_10_loc_hash("seafood", ReviewSummary, 43.66311072883239, -79.39440813687946)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,doK6H6hraPaVM6X5QE6d7Q,Bluenose Lobster,"Seafood, Restaurants, Shopping, Seafood Market...",3.0
1,D2loJREaIqjam3ct5q2Hcg,China Wok,"Chinese, Restaurants, Fast Food, Food Court",4.5
2,n-ps87_nlV56dKLD7Wep3w,Fookee Express,"Chinese, Fast Food, Restaurants",3.0
3,Bal9D1fzSgPvQPMzrvJF8g,California Thai,"Imported Food, Restaurants, Ethnic Food, Thai,...",3.5
4,PD3ganU8NAGkqkcIXYPSxQ,Fudao noodle house,"Noodles, Restaurants, Fast Food, Chinese",4.0
5,qFm-9Q4BnDSS9eWrCgO3Lw,Kimchi Tofu,"Food, Restaurants, Korean",4.5
6,LcIgUlWaJJwtOfPoPWCmBg,Souppe Shoppe,"Restaurants, Food Stands, Soup, Street Vendors...",5.0
7,crstB-H5rOfbXhV8pX0e6g,Ramen Isshin,"Noodles, Tapas/Small Plates, Food, Ramen, Japa...",4.5
8,4LoiyV3UHf-cFlqGVNBY4Q,Udon Kitanoya,"Japanese, Noodles, Restaurants, Comfort Food",3.5
9,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"Food, Ramen, Restaurants, Desserts, Japanese, ...",3.5


In [ ]:
recommendation.to_csv("sf_loc1.csv",index=False)

In [ ]:
# Scarborough Civic Centre
restaurents = top_10_loc_hash("seaFood", ReviewSummary, 43.77952047125285, -79.25780882879529)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,_xAJZOKBMPOe47p1MphB2w,Fishman Lobster Clubhouse Restaurant,"Seafood, Restaurants, Chinese, Live/Raw Food",4.0
1,aAGo2MMC2hQlG5uhPPNnjA,Su&Shi Noodle Bar,"Restaurants, Noodles, Japanese, Food Court, Su...",3.0
2,qftPq-opqmbs1BKtRj3SgA,Sure Win Cafe,"Chinese, Food Court, Restaurants, Cafes",3.0
3,dTWfATVrBfKj7Vdn0qWVWg,Flavor Cuisine,"Food Court, Chinese, Restaurants",2.5
4,Mwl79jLlxPQabA3yiSPz0A,Canton Rice Noodle,"Restaurants, Specialty Food, Food, Chinese, Co...",4.0
5,dQojkooHaWvyCor5qcAS6Q,Peking Duck Kitchen,"Specialty Food, Food, Restaurants, Chinese",3.5
6,6j67nog58__bFPqd620Rrw,JJM Dbarkadz Filipino Cuisine,"Food, Restaurants, Filipino, Food Delivery Ser...",4.0
7,3FekhBpUmAWXBCvXBhuqVQ,Da Ke Yi Snacks,"Food Court, Chinese, Restaurants",3.5
8,NE596RKQomLvdrWiM9DYWg,Golden Jade Restaurant,"Chinese, Restaurants, Food",3.0
9,PCdQZQFvcjeiiXW_W0HBKA,Kenny's Noodles,"Ethnic Food, Chinese, Specialty Food, Food, Re...",4.5


In [ ]:
recommendation.to_csv("sf_loc2.csv",index=False)

In [ ]:
# Aga Khan Museum
restaurents = top_10_loc_hash("seafood", ReviewSummary, 43.72803030610192, -79.32977026321979)
recommendation = restaurents.merge(table, on="business_id")
recommendation

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  from ipykernel import kernelapp as app


,business_id,name,categories,rating
0,6U73zDP4NuFKyOQLSigcPA,Spring China House,"Restaurants, Food Delivery Services, Food, Chi...",3.0
1,PMPMHHLe9vFzcR67iCjgLA,Faryab Kabab,"Lebanese, Kebab, Persian/Iranian, Middle Easte...",4.0
2,yHCoJZh5nx0Onr5xgMaUHg,Galleria Supermarket,"Ethical Grocery, International Grocery, Korean...",4.0
3,bwFxrxHrz9I36awGc-yjjw,TAO Northern Chinese Cuisine,"Dim Sum, Seafood, Chinese, Noodles, Food, Rest...",3.5
4,9xPK9u_AGM1sxoG2CU9psA,Golden Wok Chinese Restaurant,"Chinese, Food, Food Delivery Services, Restaur...",3.5
5,U5jgjTB442DtR-lEvV-RdQ,Green Zebra Kitchen,"Restaurants, Food Delivery Services, Food, Veg...",4.0
6,R9bFr7dADF2fosWJjgbyWQ,Nawzar Meat Market,"Middle Eastern, Meat Shops, Food, Specialty Fo...",4.5
7,LSneBLmL_mLRKjplp2BVZg,ABC Euro Delicatessen,"Delicatessen, Food, Grocery, Ethnic Grocery, D...",4.0
8,ZmnCN-U7mhuZtlWHRsSdtA,Heirloom,"Food, Food Trucks, Restaurants, Burgers, Event...",4.0
9,Nh1pLcq7TWNffAXUFUZSQw,Warung Kampung,"Indonesian, Restaurants, Food Stands, Event Pl...",4.5


In [ ]:
recommendation.to_csv("sf_loc3.csv",index=False)